### Tree collapsing
Tristan Miller

Just a short demo to show how we can adjust the tolerance for tree collapsing

In [7]:
import pandas as pd
import numpy as np
from cluster_tree import *

In [8]:
from ete3 import Tree, TreeStyle,TextFace, add_face_to_node
ts = TreeStyle()
ts.show_leaf_name = False
ts.scale =  12

In [3]:
ward_tree = pd.read_pickle("uncollapsed_tree.pkl")
c_labels = pd.read_pickle("c_labels.pkl")
doc_term = pd.read_pickle("Tfidf_Matrix.pkl")
term_list = pd.read_pickle("Feature_List.pkl")

In [4]:
tolerance=-0.3
cmeans_terms,ndocs = get_means(c_labels,doc_term[:25000,:])
collapsed_tree = collapse_label_tree(ward_tree,cmeans_terms,ndocs,tolerance)
name_tree = get_name_tree(collapsed_tree,cmeans_terms,ndocs,term_list)

In [5]:
print(str(name_tree)[-3:])

0_;


In [6]:
t=Tree(str(name_tree),format=1)
def my_layout(node):
    F = TextFace(node.name, tight_text=True)
    add_face_to_node(F, node, column=0, position="branch-right")
ts.layout_fn = my_layout

t.show(tree_style=ts)

In [8]:
Tree("(A:1,(B:1,(E:1,D:1)Internal_1:0.5)Internal_2:0.5)Root;", format=1).show()

In [9]:
print(ndocs)

[ 198.   22.   43.   60.  151.   99.   49.  306.   57.  145.  100.   97.
   75.   41.  120.   42.  113.   44.  103.   52.  164.  204.   87.   23.
  586.   73.   54.   41.   40.  122.   32.   53.   56.   93.   35.   20.
   47.   69.   74.  142.   36.  117.   48.  109.  124.  100.  125.   45.
   60.   76.   28.   69.   54.   37.  148.   41.   36.   47.   51.   55.
  152.  374.   47.   90.   70.   43.   83.   49.   60.   38.  145.   32.
   42.   64.   86.   45.   65.   63.   64.   97.  125.  101.   94.   61.
   33.   45.   20.   78.   61.   86.   51.  140.  107.   38.   55.   34.
   37.   72.   65.   57.   94.   68.   96.   76.   59.   36.  131.  113.
   37.   59.   56.   39.   65.   46.   49.   75.   52.   28.   88.   49.
   42.   51.   71.   54.   92.   80.   59.   48.  134.   56.   37.   29.
   66.   44.   66.   53.   28.   44.   43.   45.   39.   46.   34.   87.
   25.   45.   81.   33.   35.   81.   41.   55.   50.   47.   44.   93.
   84.   49.   39.   35.   48.   51.   57.   63.   

By far the largest cluster is the one that links to ISBN.  LOL